In [1]:
import duckdb as ddb

In [2]:
# install package to allow us to download data from https connection
ddb.sql("INSTALL httpfs; LOAD httpfs")

In [3]:
# create connection to duckdb database
con = ddb.connect("../air_quality.db")

In [4]:
# create new schema in db
con.execute("CREATE schema IF NOT EXISTS raw")

In [5]:
# initialize three regions duckdb needs in order to connect to an s3 bucket
con.sql("""
    SET s3_access_key_id='';
    SET s3_secret_access_key='';
    SET s3_region='';
""")

In [ ]:
# create table
con.execute("""
    CREATE TABLE IF NOT EXISTS raw.air_quality (
        location_id BIGINT,
        sensors_id BIGINT,
        "location" VARCHAR,
        "datetime" TIMESTAMP,
        lat DOUBLE,
        lon DOUBLE,
        "parameter" VARCHAR,
        units VARCHAR,
        "value" DOUBLE,
        "month" VARCHAR,
        "year" BIGINT,
        ingestion_datetime TIMESTAMP
    );

""")

In [ ]:
# read a remote csv and insert the data into our table
con.execute("""
INSERT INTO raw.air_quality
SELECT 
    location_id, 
    sensors_id, 
    "location", 
    "datetime", 
    lat, 
    lon, 
    "parameter", 
    units, 
    "value",
    "month", 
    "year",
    current_timestamp AS ingestion_datetime
FROM read_csv('s3://openaq-data-archive/records/csv.gz/locationid=2011/year=2024/month=01/*.csv.gz');
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [8]:
con.close()